In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import Image, display
from sklearn.metrics import plot_confusion_matrix
from sklearn import tree
from sklearn import utils
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn import preprocessing
from sklearn import svm



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import graphviz
import matplotlib.pyplot as plt
import sys
import string
import csv
import random
import copy
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [3]:
def gradientRegressor(clfs, lRate, estimators, depths):
    for rate in lRate:
        for trees in estimators:
            for depth in depths:
                clf = GradientBoostingRegressor(loss='ls', 
                                          learning_rate = 0.1, 
                                          n_estimators = 100, 
                                          subsample = 1.0, 
                                          criterion = 'friedman_mse', 
                                          min_samples_split = 2, 
                                          min_samples_leaf = 1, 
                                          min_weight_fraction_leaf = 0.0, 
                                          max_depth = 3, 
                                          min_impurity_decrease = 0.0, 
                                          min_impurity_split = None, 
                                          init = None, 
                                          random_state = None, 
                                          max_features = None, 
                                          alpha = 0.9, 
                                          verbose = 0, 
                                          max_leaf_nodes = None, 
                                          warm_start = False, 
                                          presort = 'deprecated', 
                                          validation_fraction = 0.1, 
                                          n_iter_no_change = None, 
                                          tol = 0.0001, 
                                          ccp_alpha = 0.0)
                clfs.append(clf)

In [4]:
def randomForest(clfs,estimators,featuresFracs,crit):
    for criter in crit:
        for estimator in estimators:
            for featuresFrac in featuresFracs:   
                  clfs.append(RandomForestClassifier(
                      n_estimators = estimator,
                      #integer, optional (default=100)
                      criterion= criter,
                      #string, optional (default=”gini”)
                      max_depth= None,
                      #integer or None, optional (default=None)
                      min_samples_split=2,
                      #int, float, optional (default=2)
                      min_samples_leaf=1,
                      #int, float, optional (default=1)
                      min_weight_fraction_leaf=0.0,
                      #float, optional (default=0.)
                      max_features= featuresFrac,
                      #int, float, string or None, optional (default=”auto”)
                      max_leaf_nodes=None,
                      #int or None, optional (default=None)
                      min_impurity_decrease=0.01,
                      #float, optional (default=0.)
                      bootstrap=True,
                      #boolean, optional (default=True)
                      oob_score=False,
                      #bool (default=False)
                      n_jobs=None,
                      #int or None, optional (default=None)
                      random_state=None,
                      #int, RandomState instance or None, optional (default=None)
                      verbose=0,
                      #int, optional (default=0)
                      warm_start=False,
                      #bool, optional (default=False)
                      ccp_alpha=0.0,
                      #non-negative float, optional (default=0.0)
                      max_samples=None
                      #int or float, default=None
                      ))

In [5]:
def decisionTree(clfs, crits, impDecreases):    
    for crit in crits:
        for imp in impDecreases:
            clf = tree.DecisionTreeClassifier(
                  criterion= "entropy",
                  splitter= crit,
                  max_depth=None,
                  min_samples_split= 2,
                  min_samples_leaf=5,
                  min_weight_fraction_leaf=0.0,
                  max_features=None,
                  random_state=None,
                  max_leaf_nodes=None,
                  min_impurity_decrease=imp,
                  class_weight=None,
                  ccp_alpha = 0.0)
            clfs.append(clf)

In [6]:
def gradientBoost(clfs, estimators):
    for estimator in estimators:
        clf = GradientBoostingClassifier(loss='deviance', 
                                   learning_rate = 0.1, 
                                   n_estimators = estimator, 
                                   subsample = 1.0, 
                                   criterion = 'friedman_mse', 
                                   min_samples_split = 2 , 
                                   min_samples_leaf = 1, 
                                   min_weight_fraction_leaf = 0.0, 
                                   max_depth = 3, 
                                   min_impurity_decrease = 0.0, 
                                   min_impurity_split = None, 
                                   init = None, 
                                   random_state = None, 
                                   max_features = None, 
                                   verbose = 0, 
                                   max_leaf_nodes = None, 
                                   warm_start = False, 
                                   presort = 'deprecated', 
                                   validation_fraction = 0.1, 
                                   n_iter_no_change = None, 
                                   tol = 0.0001, 
                                   ccp_alpha = 0.0)
        clfs.append(clf)

In [7]:
def bestNeighbors(clfs, neighbors, leafSizes):
    for neighbor in neighbors:
        for leafSize in leafSizes:
            clf = KNeighborsClassifier(n_neighbors = 5, 
                                 weights = 'uniform', 
                                 algorithm = 'auto', 
                                 leaf_size = 30, 
                                 p = 2, 
                                 metric = 'minkowski', 
                                 metric_params = None,
                                 n_jobs = None,)
            clfs.append(clf)
    

In [8]:
def printAll(value):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(value)

In [9]:
def plotImportance(forest, X, y):
    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), X.columns)
    plt.xlim([-1, X.shape[1]])
    plt.show()


In [10]:
def fillAge(X_age, y_age, X, y, train):
    #Split train and test data
    X_train, X_test, y_train, y_test = train_test_split(X_age, y_age, test_size=0.30, random_state=None)

    #Set the test values for variables in tree (criterio and minImpurity Decrease)
    clfs = []
    lRate = [0.05, 0.1, 0.15, 0.2]
    estimators = [100, 200, 500]
    depths = [3, 4, 5]
    gradientRegressor(clfs, lRate, estimators, depths)

    #Train
    testTrees = []
    minErr = 9999
    regressor = None
    for clf in clfs:
        tempClf = clf.fit(X_train, y_train)
        testTrees.append(tempClf)
        tempScore = abs(tempClf.score(X_test, y_test))
        if(tempScore < minErr):
            minErr = tempScore
            regressor = tempClf
        
    #Fill the rows with NaN in age column 
    emptyAge =  X[X['Age'] == 0]
    emptyAge = emptyAge.drop(columns = 'Age')
    if train == 1:
        emptyAge.insert(1, 'Survived', y[X['Age'] == 0])
    predAge = regressor.predict(emptyAge)
    predAge = [int(age) for age in predAge]
    X['Age'][X['Age'] == 0] = predAge

In [11]:
def trainClassifier(clfs, X, y, X_F, ids):
    
    #Split train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=None)


    scores = []
    importance = []
    #Find best classifier
    for clf in clfs:
        clf.fit(X_train, y_train)
        scores.append(clf.score(X_test, y_test))
    #Add XGBoost and SVM
    
    #XGBoost
    model = XGBClassifier()
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
    clfs.append(model)
    
    #SVM
    model = svm.SVC()
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
    clfs.append(model)
    
    #Find the best classifier for the test
    bestIndx = scores.index(max(scores))
    print('best: ' + str(bestIndx) + ' score: ' + str(max(scores)))
    best = clfs[bestIndx]
    best = best.fit(X, y)
    
    #The poorest predictions will be replaced for a prediction based on k-nearest neighbors
    #Save indices' of the poorest predictions (Almost random) if method exists
    Survived = best.predict(X_F)#Predict survivers with gradient boost classifier
    if bestIndx < 9:
        predProb = best.predict_proba(X_F) #Predict probabilities of being in 0/1 clas
        count = 0
        indices = [] 
        for row in predProb:
            dif = abs(row[0] - row[1])
            if dif < 0.15:
                indices.append(count)
            count = count + 1
        poorPred = X_F.loc[indices, :]
        print('PoorNumber: ' + str(len(indices)))
        neighbors = [] #List with different KNeighborsClassifier to select the better one 
        numNeighbors = [3, 5, 10]
        lSizes = [25, 30, 35]
        bestNeighbors(neighbors, numNeighbors, lSizes)
        best = 0
        scores = []
        for neighbor in neighbors:
            temp = neighbor.fit(X_train, y_train)
            scores.append(temp.score( X_test, y_test))

        #Find the best classifier for the test 
        best = scores.index(max(scores))
        best = neighbors[best]
        best = best.fit(X, y)
        newPred = best.predict(poorPred) #New prediction for the poorest predictions
        Survived[indices] = newPred

    #Create dataframe to submit 
    df = pd.DataFrame(columns = ['PassengerId', 'Survived']) 
    df['Survived'] = Survived
    df['PassengerId'] = ids
    df.to_csv(r'/kaggle/working/train.csv')
    
    return max(scores)

In [12]:
#Read the csv and extract the data in labels,X and y
X = pd.read_csv('/kaggle/input/titanic/train.csv')
y = X['Survived']
X.drop(columns="Survived", inplace = True)
X_testF = pd.read_csv('/kaggle/input/titanic/test.csv')

#Preprocessing to convert string data in numerical data

#Select all no numerical Data
procX = X.select_dtypes(include=[object])
procXt = X_testF.select_dtypes(include=[object])

#Fill NaN for numerical data with 0

#Train data
for column in X.columns:
    columnName = str(column)
    X[columnName].fillna(0, inplace = True)
    
#Test data
for column in X_testF.columns:
    columnName = str(column)
    X_testF[columnName].fillna(0, inplace = True)

#Drop poorest related features

#Train data
X.drop(columns = 'Name', inplace = True)
X.drop(columns = 'Ticket', inplace = True)
X.drop(columns = 'Cabin', inplace = True)
X.drop(columns = 'Fare', inplace = True)
X.drop(columns = 'PassengerId', inplace = True)

#Test data
X_testF.drop(columns = 'Name', inplace = True)
X_testF.drop(columns = 'Ticket', inplace = True)
X_testF.drop(columns = 'Cabin', inplace = True)
X_testF.drop(columns = 'Fare', inplace = True)    
pId = X_testF.drop(columns = 'PassengerId', inplace = True)


#Train data
#map Sex, age and embarked port value to a numerical value
X['Age'] = pd.cut(x = X['Age'], bins = [0, 10, 20, 30, 40, 50, 60, 80], labels = ['Infants','Teenagers','20s', '30s','40s','50s','Oldies'])
age_mapping = {'Infants': 0, 'Teenagers': 1, '20s': 2, '30s': 3, '40s': 4, '50s': 5, 'Oldies': 6}
X['Age'] = X['Age'].map(age_mapping)
X['Age'] = pd.to_numeric(X['Age'], downcast='signed')
sex_mapping = {"male": 0, "female": 1}
X['Sex'] = X['Sex'].map(sex_mapping)
port_mapping = {"S": 0, "C": 1, "Q": 2}
X['Embarked'] = X['Embarked'].map(port_mapping)

#Test data
X_testF['Age'] = pd.cut(x = X_testF['Age'], bins = [0, 10, 20, 30, 40, 50, 60, 80], labels = ['Infants','Teenagers','20s', '30s','40s','50s','Oldies'])
X_testF['Age'] = X_testF['Age'].map(age_mapping)
X_testF['Age'] = pd.to_numeric(X_testF['Age'], downcast='signed')
X_testF['Sex'] = X_testF['Sex'].map(sex_mapping)
X_testF['Embarked'] = X_testF['Embarked'].map(port_mapping)

#Fill NaN for numerical data with 0, in this case only Embarked and Age has NaN     

#Train data
X['Embarked'].fillna(0, inplace = True)
X['Age'].fillna(0, inplace = True)

#Test data 
X_testF['Embarked'].fillna(0, inplace = True)
X_testF['Age'].fillna(0, inplace = True)

#Predict age 

#train data
y_age = X['Age'][X['Age'] != 0]
y_age = y_age.tolist()      
X_age = X[X['Age'] != 0]
X_age = X_age.drop(columns = 'Age')
X_age.insert(1, 'Survived', y[X['Age'] != 0])


#test data
y_aget = X_testF['Age'][X_testF['Age'] != 0]
y_aget = y_aget.tolist()      
X_aget = X_testF[X_testF['Age'] != 0]
X_aget = X_aget.drop(columns = 'Age')

# Fill NaN age with predicted age 
fillAge(X_age, y_age, X, y, 1)
fillAge(X_aget, y_aget, X_testF, 0, 0)

final = 0
#Train and test the classifiers 
iterCount = 0

#Iterate until it find an appropiate classifier +84%
while(final < 0.84):
    #Random forest
    forests = []
    estimators = [100, 500, 1000]
    crit = ['gini', 'entropy']
    featuresFracs = [0.99]
    randomForest(forests,estimators,featuresFracs,crit)

    #Gradient Boosting Classifier
    estimators = [100, 500, 1000]
    gradientBoost(forests, estimators)
    final = trainClassifier(forests, X, y, X_testF, pId)
    iterCount = iterCount + 1
    print('IterCount: ' + str(iterCount))
    print('')
print(':)')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


best: 6 score: 0.8022388059701493
PoorNumber: 45
IterCount: 1

best: 0 score: 0.8134328358208955
PoorNumber: 42
IterCount: 2

best: 0 score: 0.8097014925373134
PoorNumber: 42
IterCount: 3

best: 10 score: 0.8470149253731343
IterCount: 4

:)
